<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2023notebooks/2023_1027Knd_Ijn_Ask_p2p_s2p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* date: 2023_1027
* author: 浅川伸一
* filename: 2023_1027Knd_Ijn_Ask_s2p_ps2.ipynb

# 符号化器‐復号化器 (encoder-decoder a.k.a seq2seq) モデルによる，単語復唱，単語産出，および単語理解処理過程の実装


see 2004Harm&Seidenberg: Computing the Meanings of Words in Reading: Cooperative Division of Labor Between Visual and Phonological Processes.

<center>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4c.svg"><br/>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2004Harm_Seidenberg_fig4d.svg"><br/>
<!-- <img src="2004Harm_Seidenberg_fig4c.svg"><br/>-->
<!-- <img src="2004Harm_Seidenberg_fig4d.svg"><br/> -->
`2004Harm&Seidenberg2004`, Figure 4 c, and d
</center>


In [ ]:
%%time
# このセルは MeCab のコンパイルに時間を要するため，実行終了まで 15 分程度かかります。
%config InlineBackend.figure_format = 'retina'
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

import sys
import os
import numpy as np
from tqdm.notebook import tqdm
import time
import datetime
import matplotlib.pyplot as plt

# ローカルと colab との相違を吸収するために必要となるライブラリをインストール
try:
    import jaconv
except ImportError:
    !pip install jaconv

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

if isColab:
    !pip install --upgrade termcolor==1.1
from termcolor import colored

# MeCab, fugashi, ipadic のインストール
if isColab:
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3==0.7 > /dev/null 2>&1
    !pip install --upgrade ipadic > /dev/null 2>&1
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1

    import subprocess
    cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                                     shell=True).communicate()[0]).decode('utf-8')

    !pip install 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download > /dev/null 2>&1
    !pip install ipadic > /dev/null 2>&1
    !pip install 'konoha[mecab]'


try:
    import ccap
except ImportError:
    !git clone https://github.com/ShinAsakawa/ccap.git
    import ccap

try:
    import RAM
except ImportError:
    !git clone https://github.com/ShinAsakawa/RAM.git
    import RAM

## 意味表現として word2vec による意味埋め込みベクトルを使う

In [ ]:
%%time
# word2vec を読み込む おそらく 1, 2 分かかる
from ccap import ccap_w2v
w2v = ccap_w2v().w2v
#w2v = ccap_w2v(isColab=False).w2v

# MeCab による yomi を輸入
from ccap.mecab_settings import yomi

## データセット Psylex71_Dataset の読み込み

In [ ]:
try:
    from kunrei import kunrei
except ImportError:
    !wget https://shinasakawa.github.io/2023notebooks/kunrei.py -O kunrei.py
    from kunrei import kunrei

# データセットとしての Psylex71_Dataset の読み込み
from RAM import Psylex71_Dataset

psylex71_ds = Psylex71_Dataset(max_words=30000)
print(f'psylex71_ds の単語数:{psylex71_ds.__len__()}')

### データセットのヒストグラム描画

In [ ]:
from RAM import draw_word_char_histgram

draw_word_char_histgram(_dict=psylex71_ds.data_dict, key='phon', title='音韻', figsize2=(8,3))

## psylex71_ds データをすべて word2vec の埋め込みベクトル行列を得る

In [ ]:
# psylex71_ds データをすべて word2vec の埋め込みベクトル行列を得る
_f = [dct['orth'] for dct in psylex71_ds.data_dict.values()]

# gensim() の `vectors_for_all()` 関数を使えば，望む語彙からなる word2vec 単語埋め込みモデルを作成できる
w2v_psylex71 = w2v.vectors_for_all(_f)

# NaN データが入っている可能性がるので変換
w2v_psylex71.vectors = np.nan_to_num(w2v_psylex71.vectors)
print(f'w2v_psylex71.vectors.shape:{w2v_psylex71.vectors.shape}')

## psylex71 データセット中の単語における w2v を表示してテストする

In [ ]:
#from tqdm.notebook import tqdm
Emb = w2v_psylex71.vectors

Wrd = input('単語を入力してください:')
color = 'blue'
while (Wrd != ""):
    if Wrd in w2v_psylex71:
        Idx = w2v_psylex71.key_to_index[Wrd]
        print(f'入力単語 Wrd:{colored(Wrd, color, attrs=["bold"])},',
              f'対応する単語番号 Idx:{colored(Idx, color, attrs=["bold"])},',
              f'w2v_psylex71.get_index({Wrd}):{colored(w2v_psylex71.get_index(Wrd), color, attrs=["bold"])}')
    else:
        print(colored(f'{Wrd} という単語はありません。','red', attrs=['bold']))
    Wrd = input('単語を入力してください (終了するには改行のみを入力):')

## PyTorch 用のデータセット定義

In [ ]:
from torch.utils.data import Dataset
import gensim

class psylex71_w2v_Dataset(Dataset):
    def __init__(self,
                 direction='s2p',  # ['s2p', 'p2s']
                 #source='seme',    # エンコーダ用 入力データ, ['orth', seme', 'phon'] のいずれか一つ
                 #target='phon',    # デコーダ用 出力データ ,  ['orth', seme', 'phon'] のいずれか一つ
                 w2v:gensim.models.keyedvectors.KeyedVectors=w2v_psylex71,
                 old_ds:RAM.dataset.Psylex71_Dataset=psylex71_ds,
                 mecab_yomi=yomi,
                ):
        self.direction = 's2p' if direction == 's2p' else direction
        self.w2v = w2v
        self.old_ds = old_ds
        self.mecab_yomi = yomi

        _wrds = []
        for idx in range(len(w2v)):
            _wrds.append(w2v.index_to_key[idx])
        self.words = _wrds
        self.W = w2v.vectors

        # 訓令式に従った日本語ローマ字表記 `kurei.py` 参照
        self.phoneme = ['<PAD>', '<SOW>', '<EOW>', '<UNK>', # 特殊トークン，純に，埋め草，語頭，語末，未知
                        'a', 'i', 'u', 'e', 'o',            # 母音
                        'a:', 'i:', 'u:', 'e:', 'o:',       # 長母音
                        'N', 'Q',                           # 撥音，拗音
                        'b', 'by', 'ch', 'd', 'dy', 'f', 'g', 'gy', 'h', 'hy', # 子音
                        'j', 'k', 'ky', 'm', 'my', 'n', 'ny',  'p', 'py', 'r', # 子音
                        'ry', 's', 'sy', 't', 'ty', 'w', 'y', 'z', 'zy']       # 子音

    def __getitem__(self,
                    idx:int,
                    direction:str=None):
        wrd = self.words[idx]
        if direction == None:
            direction = self.direction
        if direction == 'p2s':
            X = torch.LongTensor(self.wrd2phon_ids(wrd))
            y = torch.tensor(self.w2v.get_vector(idx))
        else:
            y = torch.LongTensor(self.wrd2phon_ids(wrd))
            X = torch.tensor(self.w2v.get_vector(idx))

        return X, y


    def __len__(self):
        return len(self.w2v)

    def getitem(self,
                idx:int):
        wrd = self.words[idx]
        _yomi = self.wrd2yomi(wrd)
        _yomi = kunrei(_yomi).split(' ')
        ids = [self.phoneme.index(idx) for idx in _yomi]
        return wrd, _yomi, ids

    def wrd2phon_ids(self, wrd:str)->list:
        _yomi = self.wrd2yomi(wrd)
        _yomi = kunrei(_yomi).split(' ')
        ids = [self.phoneme.index(idx) for idx in _yomi]
        ids = [self.phoneme.index('<SOW>')] + ids + [self.phoneme.index('<EOW>')]
        return ids #, _yomi

    def get_wrdidx_from_word(self, wrd:str):
        if wrd in self.words:
            wrd_idx = self.w2v.get_index(wrd)
        return wrd_idx

    def wrd2emb(self, wrd:str)->np.ndarray:
        if wrd in self.words:
            return self.w2v.get_vector(wrd)
        else:
            return None

    def wrd2wrd_idx(self, wrd:str)->int:
        if wrd in self.words:
            return self.words.index(wrd)
        else:
            return None

    def wrd_idx2wrd(self, idx:int)->str:
        if 0 <= idx and idx < len(self.words):
            return self.words[idx]
        else:
            return None

    def wrd2onehot(self, wrd:str)->np.ndarray:
        ret = np.zeros((self.W.shape[0],), dtype=np.int32)
        if wrd in self.words:
            ret[self.w2v.get_index(wrd)] = 1
            return ret
        else:
            return None

    def phon_ids2phn(self, ids:list):
        return "".join([self.phoneme[idx] for idx in ids])

    def wrd2yomi(self, wrd:str)->list:
        if wrd in self.words:
            _yomi = self.old_ds.orth2info_dict[wrd]['ヨミ']
        else:
            _yomi = self.mecab_yomi(wrd).strip().split()[0]
        return _yomi

    def wrd2info(self, wrd:str)->dict:
        if wrd in self.words:
            return self.old_ds.orth2info_dict[wrd]
        else:
            return None


_psylex71_ds = psylex71_w2v_Dataset()

# 以下確認作業
N = np.random.randint(_psylex71_ds.__len__())
wrd = _psylex71_ds.wrd_idx2wrd(N)
print(f'_Wrd:{wrd}\n',
      f'_psylex71_ds.wrd2phon_ids({wrd}):{_psylex71_ds.wrd2phon_ids(wrd)}\n',
      f'_psylex71_ds.phon_ids2phn(_psylex71_ds.wrd2phon_ids({wrd})):{_psylex71_ds.phon_ids2phn(_psylex71_ds.wrd2phon_ids(wrd))}\n',
      f'_psylex71_ds.wrd2yomi({wrd}): {_psylex71_ds.wrd2yomi(wrd)}\n',
      f'_psylex71_ds.wrd2wrd_idx({wrd}): {_psylex71_ds.wrd2wrd_idx(wrd)}\n',
      f'_psylex71_ds.wrd2info({wrd}): {_psylex71_ds.wrd2info(wrd)}')

## DataLoader の設定

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np

def my_collate_fn(batch):
    images, targets= list(zip(*batch))
    xs = list(images)
    ys = list(targets)
    return xs, ys

batch_size = 1024
dataloader = DataLoader(_psylex71_ds,
                        batch_size=batch_size,
                        shuffle=True,
                        collate_fn=my_collate_fn)

# 1 復唱モデル

<p style="font-family:serif;font-size:14pt;color:purple;font-weight:900">

PyTorch RNN モデルの実装に対する注意メモ
    
* Encoder 側のデータと Decoder 側のデータそれぞれに対して Padding の処理を行う。
* Encoder 側のデータには Padding 値として `0` で埋める。
* Decoder 側のデータをモデルの forward で使う場合には、Padding 値は `0` を埋める。
* ただし，Decoder 側のデータを教師データとして使う場合には，Padding 値には -1 を用いて，埋めることに注意。
* `nn.Embedding()` のオプションに `padding_idx=O` を付け，`CrosEntropyLoss` のオプションに `ignore_index=-1` を付ける。

</p>


## 1.1 モデルの定義

In [ ]:
# Define model
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class EncDec_w_Att(nn.Module):
    """ 注意つき符号化器‐復号化器モデル
    Bahdanau, Cho, & Bengio (2015) NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE, arXiv:1409.0473
    """
    def __init__(self,
                 enc_vocab_size:int,
                 dec_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False):
        super().__init__()

        # Encoder 側の入力トークン id を多次元ベクトルに変換
        self.encoder_emb = nn.Embedding(num_embeddings=enc_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Decoder 側の入力トークン id を多次元ベクトルに変換
        self.decoder_emb = nn.Embedding(num_embeddings=dec_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        # Encoder LSTM 本体
        self.encoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # Decoder LSTM 本体
        self.decoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # 文脈ベクトルと出力ベクトルの合成を合成する層
        bi_fact = 2 if bidirectional else 1
        self.combine_layer = nn.Linear(bi_fact * 2 * n_hid, n_hid)

        # 最終出力層
        self.out_layer = nn.Linear(n_hid, dec_vocab_size)

    def forward(self, enc_inp, dec_inp):

        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hnx, cnx) = self.encoder(enc_emb)

        dec_emb = self.decoder_emb(dec_inp)
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))

        # enc_out は (バッチサイズ，ソースの単語数，中間層の次元数)
        # ソース側 (enc_out) の各単語とターゲット側 (dec_out) の各単語との類似度を測定するため
        # 両テンソルの内積をとるため ソース側 (enc_out) の軸を入れ替え
        enc_outP = enc_out.permute(0,2,1)

        # sim の形状は (バッチサイズ, 中間層の次元数，ソースの単語数)
        sim = torch.bmm(dec_out, enc_outP)

        # sim の各次元のサイズを記録
        batch_size, dec_word_size, enc_word_size = sim.shape

        # sim に対して，ソフトマックスを行うため形状を変更
        simP = sim.reshape(batch_size * dec_word_size, enc_word_size)

        # simP のソフトマックスを用いて注意の重み alpha を算出
        alpha = F.softmax(simP,dim=1).reshape(batch_size, dec_word_size, enc_word_size)

        # 注意の重み alpha に encoder の出力を乗じて，文脈ベクトル c_t とする
        c_t = torch.bmm(alpha, enc_out)

        # torch.cat だから c_t と dec_out とで合成
        dec_out_ = torch.cat([c_t, dec_out], dim=2)
        #print(f'c_t.size():{c_t.size()}, dec_out.size():{dec_out.size()}')
        #print(f'dec_out_.size():{dec_out.size()}')
        dec_out_ = self.combine_layer(dec_out_)
        return self.out_layer(dec_out_)

n_hid = 64
n_layers = 1
bidirectional=True
# 直下行で，enc_vocab_size と dec_vocab_size を phoneme にしているので，音韻複勝課題に相当する
enc_dec = EncDec_w_Att(enc_vocab_size=len(_psylex71_ds.phoneme),
                       dec_vocab_size=len(_psylex71_ds.phoneme),
                       n_layers=n_layers,
                       bidirectional=bidirectional,
                       n_hid=n_hid).to(device)
print(enc_dec.eval())

## 1.2 訓練の実施

In [ ]:
start_time = time.time()
n_hid = 64
n_layers = 1
epochs = 10
bidirectional=True
model = EncDec_w_Att(enc_vocab_size=len(_psylex71_ds.phoneme),
                     dec_vocab_size=len(_psylex71_ds.phoneme),
                     n_layers=n_layers,
                     bidirectional=bidirectional,
                     n_hid=n_hid).to(device)


optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

model.train()
interval = int(_psylex71_ds.__len__()/batch_size) >> 2
losses = []
for epoch in range(epochs):
    i = 0
    for x, y in dataloader:
        enc_inp = pad_sequence(y, batch_first=True).to(device)
        #enc_inp = pad_sequence(y, batch_first=True).to(device)[:,:-1]
        #enc_inp = pad_sequence(y, batch_first=True).to(device)[:,1:]

        dec_inp = pad_sequence(y, batch_first=True).to(device)[:,:]
        #dec_inp = pad_sequence(y, batch_first=True).to(device)[:,:-1]
        #dec_inp = pad_sequence(y, batch_first=True).to(device)[:,1:]

        tch = pad_sequence(y, batch_first=True, padding_value=-1.0).to(device)
        #tch = tch[:,1:]

        out = model(enc_inp, dec_inp)
        loss = criterion(out[0], tch[0])
        for h in range(1,len(tch)):
            loss += criterion(out[h], tch[h])
        losses.append(loss.item()/len(x))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i % interval) == 0:
            print(f'epoch:{epoch:2d}, batch:{i:2d}, loss:{loss.item()/len(x):.3f}')

end_time = time.time()
total_time = end_time - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print(f'Training time {total_time_str}')

plt.plot(losses)
plt.title(f'epochs:{epochs}, batch_size:{batch_size}, n_hid:{n_hid}, n_layers:{n_layers}, time collapsed:{total_time_str}')
plt.show()

#outfile = "attnmt2-" + str(epoch) + ".model"
#torch.save(net.state_dict(),outfile)

## 1.3 学習結果の検証

In [ ]:
model.eval()

errors = []
isPrint = False
for N in tqdm(range(_psylex71_ds.__len__())):
    x =_psylex71_ds.__getitem__(N)[1].to(device)
    y = model(x[1:].unsqueeze(0), x[:-1].unsqueeze(0)).to('cpu')
    y_ids = np.argmax(y.squeeze(0).detach().numpy(), axis=1)[1:]
    y_phon = _psylex71_ds.phon_ids2phn(y_ids)
    grand_truth = _psylex71_ds.getitem(N)

    n_correct = np.array((y_ids == grand_truth[2]).sum())
    isOK = n_correct == len(grand_truth[2])


    color = 'grey' if isOK else 'red'
    if not isOK:
        errors.append((N,y_ids))
        if isPrint:
            print(colored((f'IDX:{N:5d}',
                           f'単語:{grand_truth[0]}',
                           f'出力:{y_phon}',
                           f'出力ID:{y_ids}',
                           f'{grand_truth[2]}'), color, attrs=['bold']))


cr = len(errors) / _psylex71_ds.__len__()
print(f'総エラー数:{len(errors)}',
      f'正解率:{(1.-cr)*100:.3f}')



# 2 産出モデル




## 2.1 モデルの定義

In [ ]:
# Define model
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class EncDec_s2p(nn.Module):
    def __init__(self,
                 sem_dim:int,
                 dec_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False,
                ):
        super().__init__()

        # 単語の意味ベクトル a.k.a 埋め込み表現 を decoder の中間層に接続するための変換層
        # 別解としては，入力層に接続する方法があるが，それはまた別実装にする
        self.enc_transform_layer = nn.Linear(in_features=sem_dim,
                                             out_features=n_hid)
        self.decoder_emb = nn.Embedding(num_embeddings=dec_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        self.decoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # 最終出力層
        self.out_layer = nn.Linear(n_hid, dec_vocab_size)

    def forward(self, enc_inp, dec_inp):

        enc_emb = self.enc_transform_layer(enc_inp)
        hnx, cnx = enc_emb.clone(), enc_emb.clone()

        dec_emb = self.decoder_emb(dec_inp)
        dec_out, (hny, cny) = self.decoder(dec_emb,(hnx, cnx))

        return self.out_layer(dec_out)

n_hid = 64
n_layers = 1
bidirectional=False
# 直下行で，enc_vocab_size と dec_vocab_size を phoneme にしているので，音韻複勝課題に相当する
enc_dec = EncDec_s2p(sem_dim=_psylex71_ds.w2v.vector_size,
                     dec_vocab_size=len(_psylex71_ds.phoneme),
                     n_layers=n_layers,
                     bidirectional=bidirectional,
                     n_hid=n_hid).to(device)
enc_dec.eval()

## 2.1 訓練の実施

In [ ]:
start_time = time.time()
n_hid = 64
n_layers = 1
epochs = 10
bidirectional=False
model = EncDec_s2p(sem_dim=_psylex71_ds.w2v.vector_size,
                   dec_vocab_size=len(_psylex71_ds.phoneme),
                   n_layers=n_layers,
                   bidirectional=bidirectional,
                   n_hid=n_hid).to(device)

optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

model.train()
interval = int(_psylex71_ds.__len__()/batch_size) >> 2
losses = []
for epoch in range(epochs):
    i = 0
    for x, y in dataloader:
        #enc_inp = torch.from_numpy(np.array(x)).to(device).unsqueeze(0)
        enc_inp= torch.tensor([_x.detach().numpy() for _x in x]).to(device)

        dec_inp = pad_sequence(y, batch_first=True).to(device)
        #dec_inp = pad_sequence(y, batch_first=True).to(device)[:,1:]
        tch = pad_sequence(y, batch_first=True, padding_value=-1.0).to(device)
        #tch = tch[:,1:]
        out = model(enc_inp, dec_inp)
        loss = criterion(out[0], tch[0])
        for h in range(1,len(tch)):
            loss += criterion(out[h], tch[h])
        losses.append(loss.item()/len(x))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i % interval) == 0:
            print(f'epoch:{epoch:2d}, batch:{i:2d}, loss:{loss.item()/len(x):.3f}')

end_time = time.time()
total_time = end_time - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print(f'Training time {total_time_str}')

plt.plot(losses)
plt.title(f'epochs:{epochs}, batch_size:{batch_size}, n_hid:{n_hid}, n_layers:{n_layers}, time collapsed:{total_time_str}')
plt.show()

In [ ]:
start_time = time.time()
n_hid = 64
#n_hid = 32
n_layers = 1
epochs = 10
bidirectional=False
model = EncDec_s2p(sem_dim=_psylex71_ds.w2v.vector_size,
                   dec_vocab_size=len(_psylex71_ds.phoneme),
                   n_layers=n_layers,
                   bidirectional=bidirectional,
                   n_hid=n_hid).to(device)

optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

model.train()
interval = int(_psylex71_ds.__len__()/batch_size) >> 2
losses = []
for epoch in range(epochs):
    i = 0
    for x, y in dataloader:
        print(type(x), len(x))
        sys.exit()
        enc_inp = torch.from_numpy(np.array(x)).float().to(device).unsqueeze(0)
        #enc_inp = torch.from_numpy(np.array(x)).to(device).unsqueeze(0)
        dec_inp = pad_sequence(y, batch_first=True).to(device)[:,1:]
        tch = pad_sequence(y, batch_first=True, padding_value=-1.0).to(device)
        tch = tch[:,1:]
        out = model(enc_inp, dec_inp)
        loss = criterion(out[0], tch[0])
        for h in range(1,len(tch)):
            loss += criterion(out[h], tch[h])
        losses.append(loss.item()/len(x))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        if (i % interval) == 0:
            print(f'epoch:{epoch:2d}, batch:{i:2d}, loss:{loss.item()/len(x):.3f}')

end_time = time.time()
total_time = end_time - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print(f'Training time {total_time_str}')

plt.plot(losses)
plt.title(f'epochs:{epochs}, batch_size:{batch_size}, n_hid:{n_hid}, n_layers:{n_layers}, time collapsed:{total_time_str}')
plt.show()

#outfile = "attnmt2-" + str(epoch) + ".model"
#torch.save(net.state_dict(),outfile)

## 2.2 学習結果の評価

In [ ]:
errors = []
model.eval()
for N in range(_psylex71_ds.__len__()):
#for N in np.random.permutation(_psylex71_ds.__len__())[:100]:
    x, y = _psylex71_ds.__getitem__(N)
    enc_inp = torch.from_numpy(np.array(x)).to(device).unsqueeze(0)
    enc_emb = model.enc_transform_layer(enc_inp)
    hnx, cnx = enc_emb.clone(), enc_emb.clone()
    dec_inp = y
    dec_emb = model.decoder_emb(dec_inp)
    dec_out, (hny, cny) = model.decoder(dec_emb,(hnx, cnx))
    dec_out = model.out_layer(dec_out)
    y_ids = np.argmax(dec_out.detach().numpy(),axis=1)

    n_correct = np.array((y_ids[1:-1] == _psylex71_ds.getitem(N)[2]).sum())
    isOK = n_correct == len(_psylex71_ds.getitem(N)[2])
    color = 'grey' if isOK else 'red'

    if not isOK:
        errors.append((N,y_ids))
        print(colored((f'{N:05d}', #y_ids,
                       "".join(p for p in _psylex71_ds.phon_ids2phn(y_ids[1:-1]))),color,attrs=["bold"]), end=" ")
        print(_psylex71_ds.getitem(N))

cr = len(errors) / _psylex71_ds.__len__()
print(f'総エラー数:{len(errors)}',
      f'正解率:{(1.-cr)*100:.3f}')


# 3 理解モデル

## 3.1 モデルの定義


In [ ]:
# Define model
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class EncDec_p2s(nn.Module):
    def __init__(self,
                 sem_dim:int,
                 enc_vocab_size:int,
                 n_hid:int,
                 n_layers:int=2,
                 bidirectional:bool=False):
        super().__init__()

        self.encoder_emb = nn.Embedding(num_embeddings=enc_vocab_size,
                                        embedding_dim=n_hid,
                                        padding_idx=0)

        self.encoder = nn.LSTM(input_size=n_hid,
                               hidden_size=n_hid,
                               num_layers=n_layers,
                               batch_first=True,
                               bidirectional=bidirectional)

        # 文脈ベクトルと出力ベクトルの合成を合成する層
        bi_fact = 2 if bidirectional else 1
        self.out_layer = nn.Linear(in_features=n_hid * bi_fact,
                                   out_features=sem_dim)

    def forward(self, enc_inp):
        enc_emb = self.encoder_emb(enc_inp)
        enc_out, (hnx, cnx) = self.encoder(enc_emb)
        dec_out = self.out_layer(hnx)
        return dec_out

n_hid = 64
n_layers = 1
bidirectional=False
enc_dec = EncDec_p2s(sem_dim=_psylex71_ds.w2v.vector_size,
                     enc_vocab_size=len(_psylex71_ds.phoneme),
                     n_layers=n_layers,
                     bidirectional=bidirectional,
                     n_hid=n_hid).to(device)
enc_dec.eval()